In [ ]:
using PMP, DataFrames, Distributions, Gadfly, Cairo, Fontconfig, Statistics, GLM, StatsBase


---

# Distribution A

## Parameters b = 50, $\mu$ = 1/2, $\nu$ = 4 ($\alpha$ = 2, $\beta$ = 2)
Creating samples

In [ ]:
d1 = PearsonType1b(50, 2, 2)
taille = [100, 500, 1000, 2000, 5000, 8000];

In [ ]:
nb = 100
sample_a = Array{Float64, 2}(undef, nb, taille[1])
sample_b = Array{Float64, 2}(undef, nb, taille[2])
sample_c = Array{Float64, 2}(undef, nb, taille[3])
sample_d = Array{Float64, 2}(undef, nb, taille[4])
sample_e = Array{Float64, 2}(undef, nb, taille[5])
sample_f = Array{Float64, 2}(undef, nb, taille[6])
sample_names = [sample_a, sample_b, sample_c, sample_d, sample_e, sample_f]

for i in 1:nb
    for j in 1:length(sample_names)
        sample_names[j][i, :] = rand(d1, taille[j])
    end
end

### Mixed method

In [ ]:
params_a = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_b = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_c = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_d = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_e = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_f = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
names = [params_a, params_b, params_c, params_d, params_e, params_f]

for i in 1:nb
    for j in 1:length(taille)
        fit = getinitialvalues(PearsonType1c, sample_names[j][i, :])
        estimation = DataFrame(b=fit[1], μ=fit[2], ν=fit[3])
        append!(names[j], estimation)
    end
    print(i)
end

df1m = DataFrame(taille = taille[1], b = mean(params_a.b), μ = mean(params_a.μ), ν = mean(params_a.ν))

for j in 2:length(taille)
    append!(df1m, DataFrame(taille=taille[j], b=mean(names[j].b),  μ=mean(names[j].μ), ν=mean(names[j].ν)))
end

quantile_b = Array{Float64, 2}(undef, 2, 6)
quantile_μ = Array{Float64, 2}(undef, 2, 6)
quantile_ν = Array{Float64, 2}(undef, 2, 6)

for j in 1:length(taille)
    quantile_b[:,j] = quantile(names[j].b, [.025, .975])
    quantile_μ[:,j] = quantile(names[j].μ, [.025, .975])
    quantile_ν[:,j] = quantile(names[j].ν, [.025, .975])
end


df1m.bq2_5 = quantile_b[1,:]
df1m.bq97_5 = quantile_b[2,:]

df1m.μq2_5 = quantile_μ[1,:]
df1m.μq97_5 = quantile_μ[2,:]

df1m.νq2_5 = quantile_ν[1,:]
df1m.νq97_5 = quantile_ν[2,:]

df1m

In [ ]:
df1m_corr = append!(params_a, params_b)
append!(df1m_corr, params_c)
append!(df1m_corr, params_d)
append!(df1m_corr, params_e)
append!(df1m_corr, params_f);

In [ ]:
p1 = plot(df1m, x=:taille, y=:b, Geom.line, Geom.point, 
            layer(yintercept=[50], Theme(default_color=color("black")), Geom.hline(style=:dot)), 
            layer(x=:taille, y=:bq2_5, Geom.line, Theme(default_color=color("red"))),
            layer(x=:taille, y=:bq97_5, Geom.line, Theme(default_color=color("red"))),
    Guide.Title("Mixed"), Guide.xlabel("Size"),
    Coord.cartesian(ymin=44, ymax=60),
    style(major_label_font_size=16pt, minor_label_font_size=14pt))
p1

### Maximum likelihood estimation

In [ ]:
params_a = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_b = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_c = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_d = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_e = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_f = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
names = [params_a, params_b, params_c, params_d, params_e, params_f]

for i in 1:nb
    for j in 1:length(taille)
        fit = params(fit_mle(PearsonType1c, sample_names[j][i, :]))
        estimation = DataFrame(b=fit[1], μ=fit[2], ν=fit[3])
        append!(names[j], estimation)
    end
    print(i)
end

df1mle = DataFrame(taille = taille[1], b = mean(params_a.b), μ = mean(params_a.μ), ν = mean(params_a.ν))

for j in 2:length(taille)
    append!(df1mle, DataFrame(taille=taille[j], b=mean(names[j].b),  μ=mean(names[j].μ), ν=mean(names[j].ν)))
end

quantile_b = Array{Float64, 2}(undef, 2, 6)
quantile_μ = Array{Float64, 2}(undef, 2, 6)
quantile_ν = Array{Float64, 2}(undef, 2, 6)

for j in 1:length(taille)
    quantile_b[:,j] = quantile(names[j].b, [.025, .975])
    quantile_μ[:,j] = quantile(names[j].μ, [.025, .975])
    quantile_ν[:,j] = quantile(names[j].ν, [.025, .975])
end


df1mle.bq2_5 = quantile_b[1,:]
df1mle.bq97_5 = quantile_b[2,:]

df1mle.μq2_5 = quantile_μ[1,:]
df1mle.μq97_5 = quantile_μ[2,:]

df1mle.νq2_5 = quantile_ν[1,:]
df1mle.νq97_5 = quantile_ν[2,:]

df1mle

In [ ]:
df1mle_corr = append!(params_a, params_b)
append!(df1mle_corr, params_c)
append!(df1mle_corr, params_d)
append!(df1mle_corr, params_e)
append!(df1mle_corr, params_f);

In [ ]:
p1mle = plot(df1mle, x=:taille, y=:b, Geom.line, Geom.point, 
            layer(yintercept=[50], Theme(default_color=color("black")), Geom.hline(style=:dot)), 
            layer(x=:taille, y=:bq2_5, Geom.line, Theme(default_color=color("red"))),
            layer(x=:taille, y=:bq97_5, Geom.line, Theme(default_color=color("red"))),
    Guide.Title("MLE"), Guide.xlabel("Size"),
    Coord.cartesian(ymin=44, ymax=60),
    style(major_label_font_size=16pt, minor_label_font_size=14pt))
p1mle

In [ ]:
t1 = title(hstack(p1, p1mle), "Distribution A")

### Parameter correlation

In [ ]:
# Mixed
lm1m_aa = lm(@formula(μ ~ b), df1m_corr) # between b and μ
lm1m_bb = lm(@formula(ν ~ b), df1m_corr) # between b and ν

# MLE
lm1mle_aa = lm(@formula(μ ~ b), df1mle_corr) # between b and μ
lm1mle_bb = lm(@formula(ν ~ b), df1mle_corr) # between b and ν

In [ ]:
bμ_distrA = plot(df1m_corr, x=:b, y=:μ, Geom.point, color=["Mixed"],
            layer(df1mle_corr, x=:b, y=:μ, Geom.point, color=["MLE"]),
            intercept=[coef(lm1m_aa)[1], coef(lm1mle_aa)[1]], 
            slope=[coef(lm1m_aa)[2], coef(lm1mle_aa)[2]], 
            Geom.abline(color=["skyblue1", "yellow3"], style=:dash), Guide.colorkey("Method"),
            style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt,
                    key_title_font_size=16pt, key_label_font_size=14pt)) 
bμ_distrA

In [ ]:
bν_distrA = plot(df1m_corr, x=:b, y=:ν, Geom.point, color=["Mixed"],
            layer(df1mle_corr, x=:b, y=:ν, Geom.point, color=["MLE"]),
            intercept=[coef(lm1m_bb)[1], coef(lm1mle_bb)[1]], 
            slope=[coef(lm1m_bb)[2], coef(lm1mle_bb)[2]], 
            Geom.abline(color=["skyblue1", "yellow3"], style=:dash), Guide.colorkey("Method"),
            style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt,
                    key_title_font_size=16pt, key_label_font_size=14pt)) 
bν_distrA

In [ ]:
corr_distrA = title(hstack(bμ_distrA, bν_distrA), "Distribution A")


---
# Distribution B

## Parameters : b = 50, $\mu$ = 1/4, $\nu$ = 4 ($\alpha$ = 1, $\beta$ = 3)
Creating samples

In [ ]:
d2 = PearsonType1b(50, 1, 3)

sample_a = Array{Float64, 2}(undef, nb, taille[1])
sample_b = Array{Float64, 2}(undef, nb, taille[2])
sample_c = Array{Float64, 2}(undef, nb, taille[3])
sample_d = Array{Float64, 2}(undef, nb, taille[4])
sample_e = Array{Float64, 2}(undef, nb, taille[5])
sample_f = Array{Float64, 2}(undef, nb, taille[6])
sample_names = [sample_a, sample_b, sample_c, sample_d, sample_e, sample_f]

for i in 1:nb
    for j in 1:length(sample_names)
        sample_names[j][i, :] = rand(d2, taille[j])
    end
end

### Mixed method

In [ ]:
params_a = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_b = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_c = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_d = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_e = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_f = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
names = [params_a, params_b, params_c, params_d, params_e, params_f]

for i in 1:nb
    for j in 1:length(taille)
        fit = getinitialvalues(PearsonType1c, sample_names[j][i, :])
        estimation = DataFrame(b=fit[1], μ=fit[2], ν=fit[3])
        append!(names[j], estimation)
    end
    print(i)
end

df2m = DataFrame(taille = taille[1], b = mean(params_a.b), μ = mean(params_a.μ), ν = mean(params_a.ν))

for j in 2:length(taille)
    append!(df2m, DataFrame(taille=taille[j], b=mean(names[j].b),  μ=mean(names[j].μ), ν=mean(names[j].ν)))
end

quantile_b = Array{Float64, 2}(undef, 2, 6)
quantile_μ = Array{Float64, 2}(undef, 2, 6)
quantile_ν = Array{Float64, 2}(undef, 2, 6)

for j in 1:length(taille)
    quantile_b[:,j] = quantile(names[j].b, [.025, .975])
    quantile_μ[:,j] = quantile(names[j].μ, [.025, .975])
    quantile_ν[:,j] = quantile(names[j].ν, [.025, .975])
end


df2m.bq2_5 = quantile_b[1,:]
df2m.bq97_5 = quantile_b[2,:]

df2m.μq2_5 = quantile_μ[1,:]
df2m.μq97_5 = quantile_μ[2,:]

df2m.νq2_5 = quantile_ν[1,:]
df2m.νq97_5 = quantile_ν[2,:]

df2m

In [ ]:
df2m_corr = append!(params_a, params_b)
append!(df2m_corr, params_c)
append!(df2m_corr, params_d)
append!(df2m_corr, params_e)
append!(df2m_corr, params_f);

In [ ]:
p2 = plot(df2m, x=:taille, y=:b, Geom.line, Geom.point, 
            layer(yintercept=[50], Theme(default_color=color("black")), Geom.hline(style=:dot)), 
            layer(x=:taille, y=:bq2_5, Geom.line, Theme(default_color=color("red"))),
            layer(x=:taille, y=:bq97_5, Geom.line, Theme(default_color=color("red"))),
    Guide.Title("Mixed"), Guide.xlabel("Size"),
    Coord.cartesian(ymin=30, ymax=70),
    style(major_label_font_size=16pt, minor_label_font_size=14pt))
p2

### Maximum likelihood estimation

In [ ]:
params_a = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_b = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_c = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_d = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_e = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_f = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
names = [params_a, params_b, params_c, params_d, params_e, params_f]

for i in 1:nb
    for j in 1:length(taille)
        fit = params(fit_mle(PearsonType1c, sample_names[j][i, :]))
        estimation = DataFrame(b=fit[1], μ=fit[2], ν=fit[3])
        append!(names[j], estimation)
    end
    print(i)
end

df2mle = DataFrame(taille = taille[1], b = mean(params_a.b), μ = mean(params_a.μ), ν = mean(params_a.ν))

for j in 2:length(taille)
    append!(df2mle, DataFrame(taille=taille[j], b=mean(names[j].b),  μ=mean(names[j].μ), ν=mean(names[j].ν)))
end

quantile_b = Array{Float64, 2}(undef, 2, 6)
quantile_μ = Array{Float64, 2}(undef, 2, 6)
quantile_ν = Array{Float64, 2}(undef, 2, 6)

for j in 1:length(taille)
    quantile_b[:,j] = quantile(names[j].b, [.025, .975])
    quantile_μ[:,j] = quantile(names[j].μ, [.025, .975])
    quantile_ν[:,j] = quantile(names[j].ν, [.025, .975])
end


df2mle.bq2_5 = quantile_b[1,:]
df2mle.bq97_5 = quantile_b[2,:]

df2mle.μq2_5 = quantile_μ[1,:]
df2mle.μq97_5 = quantile_μ[2,:]

df2mle.νq2_5 = quantile_ν[1,:]
df2mle.νq97_5 = quantile_ν[2,:]

df2mle

In [ ]:
df2mle_corr = append!(params_a, params_b)
append!(df2mle_corr, params_c)
append!(df2mle_corr, params_d)
append!(df2mle_corr, params_e)
append!(df2mle_corr, params_f);

In [ ]:
p2mle = plot(df2mle, x=:taille, y=:b, Geom.line, Geom.point, 
            layer(yintercept=[50], Theme(default_color=color("black")), Geom.hline(style=:dot)), 
            layer(x=:taille, y=:bq2_5, Geom.line, Theme(default_color=color("red"))),
            layer(x=:taille, y=:bq97_5, Geom.line, Theme(default_color=color("red"))),
    Guide.Title("MLE"), Guide.xlabel("Size"),
    Coord.cartesian(ymin=30, ymax=70),
    style(major_label_font_size=16pt, minor_label_font_size=14pt))
p2mle

In [ ]:
t2 = title(hstack(p2, p2mle), "Distribution B")

### Parameter correlation

In [ ]:
# Mixed
lm2m_aa = lm(@formula(μ ~ b), df2m_corr) # between b and μ
lm2m_bb = lm(@formula(ν ~ b), df2m_corr) # between b and ν

# MLE
lm2mle_aa = lm(@formula(μ ~ b), df2mle_corr) # between b and μ
lm2mle_bb = lm(@formula(ν ~ b), df2mle_corr) # between b and ν


In [ ]:
bμ_distrB = plot(df2m_corr, x=:b, y=:μ, Geom.point, color=["Mixed"],
            layer(df2mle_corr, x=:b, y=:μ, Geom.point, color=["MLE"]),
            intercept=[coef(lm2m_aa)[1], coef(lm2mle_aa)[1]], 
            slope=[coef(lm2m_aa)[2], coef(lm2mle_aa)[2]], 
            Geom.abline(color=["skyblue1", "yellow3"], style=:dash), Guide.colorkey("Method"),
            style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt,
                    key_title_font_size=16pt, key_label_font_size=14pt)) 
bμ_distrB

In [ ]:
bν_distrB = plot(df2m_corr, x=:b, y=:ν, Geom.point, color=["Mixed"],
            layer(df2mle_corr, x=:b, y=:ν, Geom.point, color=["MLE"]),
            intercept=[coef(lm2m_bb)[1], coef(lm2mle_bb)[1]], 
            slope=[coef(lm2m_bb)[2], coef(lm2mle_bb)[2]], 
            Geom.abline(color=["skyblue1", "yellow3"], style=:dash), Guide.colorkey("Method"),
            style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt,
                    key_title_font_size=16pt, key_label_font_size=14pt)) 
bν_distrB

In [ ]:
corr_distrB = title(hstack(bμ_distrB, bν_distrB), "Distribution B")


---
# Distribution C
## Parameters : b = 50, $\mu$ = 0.1/6.1, $\nu$ = 6.1 ($\alpha$ = 0.1, $\beta$ = 6)
Creating samples

In [ ]:
d3 = PearsonType1b(50, 0.1, 6)

sample_a = Array{Float64, 2}(undef, nb, taille[1])
sample_b = Array{Float64, 2}(undef, nb, taille[2])
sample_c = Array{Float64, 2}(undef, nb, taille[3])
sample_d = Array{Float64, 2}(undef, nb, taille[4])
sample_e = Array{Float64, 2}(undef, nb, taille[5])
sample_f = Array{Float64, 2}(undef, nb, taille[6])
sample_names = [sample_a, sample_b, sample_c, sample_d, sample_e, sample_f]

for i in 1:nb
    for j in 1:length(sample_names)
        sample_names[j][i, :] = rand(d3, taille[j])
    end
end

### Mixed method

In [ ]:
params_a = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_b = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_c = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_d = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_e = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_f = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
names = [params_a, params_b, params_c, params_d, params_e, params_f]

for i in 1:nb
    for j in 1:length(taille)
        fit = getinitialvalues(PearsonType1c, sample_names[j][i, :])
        estimation = DataFrame(b=fit[1], μ=fit[2], ν=fit[3])
        append!(names[j], estimation)
    end
    print(i)
end

df3m = DataFrame(taille = taille[1], b = mean(params_a.b), μ = mean(params_a.μ), ν = mean(params_a.ν))

for j in 2:length(taille)
    append!(df3m, DataFrame(taille=taille[j], b=mean(names[j].b),  μ=mean(names[j].μ), ν=mean(names[j].ν)))
end

quantile_b = Array{Float64, 2}(undef, 2, 6)
quantile_μ = Array{Float64, 2}(undef, 2, 6)
quantile_ν = Array{Float64, 2}(undef, 2, 6)

for j in 1:length(taille)
    quantile_b[:,j] = quantile(names[j].b, [.025, .975])
    quantile_μ[:,j] = quantile(names[j].μ, [.025, .975])
    quantile_ν[:,j] = quantile(names[j].ν, [.025, .975])
end


df3m.bq2_5 = quantile_b[1,:]
df3m.bq97_5 = quantile_b[2,:]

df3m.μq2_5 = quantile_μ[1,:]
df3m.μq97_5 = quantile_μ[2,:]

df3m.νq2_5 = quantile_ν[1,:]
df3m.νq97_5 = quantile_ν[2,:]

df3m

In [ ]:
df3m_corr = append!(params_a, params_b)
append!(df3m_corr, params_c)
append!(df3m_corr, params_d)
append!(df3m_corr, params_e)
append!(df3m_corr, params_f);

In [ ]:
p3m = plot(df3m, x=:taille, y=:b, Geom.line, Geom.point, 
            layer(yintercept=[50], Theme(default_color=color("black")), Geom.hline(style=:dot)), 
            layer(x=:taille, y=:bq2_5, Geom.line, Theme(default_color=color("red"))),
            layer(x=:taille, y=:bq97_5, Geom.line, Theme(default_color=color("red"))),
    Guide.Title("Mixed"), Guide.xlabel("Size"),
    Coord.cartesian(ymin=0, ymax=200),
    style(major_label_font_size=16pt, minor_label_font_size=14pt))
p3m

### Maximum likelihood estimation

In [ ]:
params_a = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_b = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_c = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_d = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_e = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_f = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
names = [params_a, params_b, params_c, params_d, params_e, params_f]

for i in 1:nb
    for j in 1:length(taille)
        fit = params(fit_mle(PearsonType1c, sample_names[j][i, :]))
        estimation = DataFrame(b=fit[1], μ=fit[2], ν=fit[3])
        append!(names[j], estimation)
    end
    print(i)
end

df3mle = DataFrame(taille = taille[1], b = mean(params_a.b), μ = mean(params_a.μ), ν = mean(params_a.ν))

for j in 2:length(taille)
    append!(df3mle, DataFrame(taille=taille[j], b=mean(names[j].b),  μ=mean(names[j].μ), ν=mean(names[j].ν)))
end

quantile_b = Array{Float64, 2}(undef, 2, 6)
quantile_μ = Array{Float64, 2}(undef, 2, 6)
quantile_ν = Array{Float64, 2}(undef, 2, 6)

for j in 1:length(taille)
    quantile_b[:,j] = quantile(names[j].b, [.025, .975])
    quantile_μ[:,j] = quantile(names[j].μ, [.025, .975])
    quantile_ν[:,j] = quantile(names[j].ν, [.025, .975])
end


df3mle.bq2_5 = quantile_b[1,:]
df3mle.bq97_5 = quantile_b[2,:]

df3mle.μq2_5 = quantile_μ[1,:]
df3mle.μq97_5 = quantile_μ[2,:]

df3mle.νq2_5 = quantile_ν[1,:]
df3mle.νq97_5 = quantile_ν[2,:]

df3mle

In [ ]:
df3mle_corr = append!(params_a, params_b)
append!(df3mle_corr, params_c)
append!(df3mle_corr, params_d)
append!(df3mle_corr, params_e)
append!(df3mle_corr, params_f);

In [ ]:
p3mle = plot(df3mle, x=:taille, y=:b, Geom.line, Geom.point, 
            layer(yintercept=[50], Theme(default_color=color("black")), Geom.hline(style=:dot)), 
            layer(x=:taille, y=:bq2_5, Geom.line, Theme(default_color=color("red"))),
            layer(x=:taille, y=:bq97_5, Geom.line, Theme(default_color=color("red"))),
    Guide.Title("MLE"), Guide.xlabel("Size"),
    style(major_label_font_size=16pt, minor_label_font_size=14pt))
p3mle

In [ ]:
t3 = title(hstack(p3m, p3mle), "Distribution C")

### Parameter correlation

In [ ]:
# Mixed
lm3m_aa = lm(@formula(μ ~ b), df3m_corr) # between b and μ
lm3m_bb = lm(@formula(ν ~ b), df3m_corr) # between b and ν

# MLE
lm3mle_aa = lm(@formula(μ ~ b), df3mle_corr)
lm3mle_bb = lm(@formula(ν ~ b), df3mle_corr)

In [ ]:
bμmle_distrC = plot(df3m_corr, x=:b, y=:μ, Geom.point, color=["Mixed"],
                layer(df3mle_corr, x=:b, y=:μ, Geom.point, color=["MLE"]),
            Coord.cartesian(xmax=200, ymax=0.3), Guide.colorkey("Method"),
            style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt,
                    key_title_font_size=16pt, key_label_font_size=14pt)) 
bμmle_distrC

bνmle_distrC = plot(df3m_corr, x=:b, y=:ν, Geom.point, color=["Mixed"],
                layer(df3mle_corr, x=:b, y=:ν, Geom.point, color=["MLE"]),
            intercept=[coef(lm3m_bb)[1], coef(lm3mle_bb)[1]], 
            slope=[coef(lm3m_bb)[2], coef(lm3mle_bb)[2]], 
            Coord.cartesian(xmax=500, ymax=100),
            Geom.abline(color=["skyblue1", "yellow3"], style=:dash), Guide.colorkey("Method"),
            style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt,
                    key_title_font_size=16pt, key_label_font_size=14pt)) 
bνmle_distrC

corr_distrC = title(hstack(bμmle_distrC, bνmle_distrC), "Distribution C")


---
# Distribution D
## Paramètres : b = 50, $\mu$ = 0.1/15.1, $\nu$ = 15.1 ($\alpha$ = 0.1, $\beta$ = 15)
Creating samples

In [ ]:
d4 = PearsonType1b(50, 0.1, 15)

sample_a = Array{Float64, 2}(undef, nb, taille[1])
sample_b = Array{Float64, 2}(undef, nb, taille[2])
sample_c = Array{Float64, 2}(undef, nb, taille[3])
sample_d = Array{Float64, 2}(undef, nb, taille[4])
sample_e = Array{Float64, 2}(undef, nb, taille[5])
sample_f = Array{Float64, 2}(undef, nb, taille[6])
sample_names = [sample_a, sample_b, sample_c, sample_d, sample_e, sample_f]

for i in 1:nb
    for j in 1:length(sample_names)
        sample_names[j][i, :] = rand(d4, taille[j])
    end
end

### Mixed method

In [ ]:
params_a = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_b = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_c = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_d = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_e = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_f = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
names = [params_a, params_b, params_c, params_d, params_e, params_f]

for i in 1:nb
    for j in 1:length(taille)
        fit = getinitialvalues(PearsonType1c, sample_names[j][i, :])
        estimation = DataFrame(b=fit[1], μ=fit[2], ν=fit[3])
        append!(names[j], estimation)
    end
    print(i)
end

df4m = DataFrame(taille = taille[1], b = mean(params_a.b), μ = mean(params_a.μ), ν = mean(params_a.ν))

for j in 2:length(taille)
    append!(df4m, DataFrame(taille=taille[j], b=mean(names[j].b),  μ=mean(names[j].μ), ν=mean(names[j].ν)))
end

quantile_b = Array{Float64, 2}(undef, 2, 6)
quantile_μ = Array{Float64, 2}(undef, 2, 6)
quantile_ν = Array{Float64, 2}(undef, 2, 6)

for j in 1:length(taille)
    quantile_b[:,j] = quantile(names[j].b, [.025, .975])
    quantile_μ[:,j] = quantile(names[j].μ, [.025, .975])
    quantile_ν[:,j] = quantile(names[j].ν, [.025, .975])
end


df4m.bq2_5 = quantile_b[1,:]
df4m.bq97_5 = quantile_b[2,:]

df4m.μq2_5 = quantile_μ[1,:]
df4m.μq97_5 = quantile_μ[2,:]

df4m.νq2_5 = quantile_ν[1,:]
df4m.νq97_5 = quantile_ν[2,:]

df4m

In [ ]:
df4m_corr = append!(params_a, params_b)
append!(df4m_corr, params_c)
append!(df4m_corr, params_d)
append!(df4m_corr, params_e)
append!(df4m_corr, params_f);

In [ ]:
p4 = plot(df4m, x=:taille, y=:b, Geom.line, Geom.point, 
            layer(yintercept=[50], Theme(default_color=color("black")), Geom.hline(style=:dot)), 
            layer(x=:taille, y=:bq2_5, Geom.line, Theme(default_color=color("red"))),
            layer(x=:taille, y=:bq97_5, Geom.line, Theme(default_color=color("red"))),
    Guide.Title("Mixed"), Guide.xlabel("Size"),
    Coord.cartesian(ymin=0, ymax=300),
    style(major_label_font_size=16pt, minor_label_font_size=14pt))
p4

### Maximum likelihood estimation

In [ ]:
params_a = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_b = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_c = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_d = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_e = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
params_f = DataFrame(b = Float64[], μ = Float64[], ν = Float64[])
names = [params_a, params_b, params_c, params_d, params_e, params_f]

for i in 1:nb
    for j in 1:length(taille)
        fit = params(fit_mle(PearsonType1c, sample_names[j][i, :]))
        estimation = DataFrame(b=fit[1], μ=fit[2], ν=fit[3])
        append!(names[j], estimation)
    end
    print(i)
end

df4mle = DataFrame(taille = taille[1], b = mean(params_a.b), μ = mean(params_a.μ), ν = mean(params_a.ν))

for j in 2:length(taille)
    append!(df4mle, DataFrame(taille=taille[j], b=mean(names[j].b),  μ=mean(names[j].μ), ν=mean(names[j].ν)))
end

quantile_b = Array{Float64, 2}(undef, 2, 6)
quantile_μ = Array{Float64, 2}(undef, 2, 6)
quantile_ν = Array{Float64, 2}(undef, 2, 6)

for j in 1:length(taille)
    quantile_b[:,j] = quantile(names[j].b, [.025, .975])
    quantile_μ[:,j] = quantile(names[j].μ, [.025, .975])
    quantile_ν[:,j] = quantile(names[j].ν, [.025, .975])
end


df4mle.bq2_5 = quantile_b[1,:]
df4mle.bq97_5 = quantile_b[2,:]

df4mle.μq2_5 = quantile_μ[1,:]
df4mle.μq97_5 = quantile_μ[2,:]

df4mle.νq2_5 = quantile_ν[1,:]
df4mle.νq97_5 = quantile_ν[2,:]

df4mle

In [ ]:
df4mle_corr = append!(params_a, params_b)
append!(df4mle_corr, params_c)
append!(df4mle_corr, params_d)
append!(df4mle_corr, params_e)
append!(df4mle_corr, params_f);

In [ ]:
p4mle = plot(df4mle, x=:taille, y=:b, Geom.line, Geom.point, 
            layer(yintercept=[50], Theme(default_color=color("black")), Geom.hline(style=:dot)), 
            layer(x=:taille, y=:bq2_5, Geom.line, Theme(default_color=color("red"))),
            layer(x=:taille, y=:bq97_5, Geom.line, Theme(default_color=color("red"))),
    Guide.Title("MLE"), Guide.xlabel("Size"),
    style(major_label_font_size=16pt, minor_label_font_size=14pt))
p4mle

In [ ]:
t4 = title(hstack(p4, p4mle), "Distribution D")

### Parameter correlation

In [ ]:
# Mixed
lm4m_aa = lm(@formula(μ ~ b), df4m_corr)
lm4m_bb = lm(@formula(ν ~ b), df4m_corr)

# MLE
lm4mle_aa = lm(@formula(μ ~ b), df4mle_corr)
lm4mle_bb = lm(@formula(ν ~ b), df4mle_corr)

In [ ]:
bμmle_distrD = plot(df4m_corr, x=:b, y=:μ, Geom.point, color=["Mixed"],
    layer(df4mle_corr, x=:b, y=:μ, Geom.point, color=["MLE"]),
    Coord.cartesian(ymax=0.4, xmax=1000, xmin=0), Guide.colorkey("Méthode"),
            style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt,
                    key_title_font_size=16pt, key_label_font_size=14pt)) 
bμmle_distrD

bνmle_distrD = plot(df4m_corr, x=:b, y=:ν, Geom.point, color=["Mixed"],
            layer(df4mle_corr, x=:b, y=:ν, Geom.point, color=["MLE"]),
            intercept=[coef(lm4m_bb)[1], coef(lm4mle_bb)[1]], 
            slope=[coef(lm4m_bb)[2], coef(lm4mle_bb)[2]], 
            Geom.abline(color=["skyblue1", "yellow3"], style=:dash), Guide.colorkey("Method"),
            Coord.cartesian(ymax=1000, xmax=500),
            style(major_label_font_size=16pt, minor_label_font_size=14pt, highlight_width = 0.05pt,
                    key_title_font_size=16pt, key_label_font_size=14pt)) 
bνmle_distrD

corr_distrDmle = title(hstack(bμmle_distrD, bνmle_distrD), "Distribution D")